# Selección Asistida, Daniel Vallejo Aldana Optimización 1

In [88]:
import numpy as np
from numpy import linalg as la 
from functools import partial

In [89]:
def LoadHist(source:str):
    #Leer número de bins
    with open(source, 'r') as f:
        line = (f.readline()).split()
    nbins = int(line[0])
    #Leer los datos en la matriz de forma apropiada
    hist = np.genfromtxt(source, skip_header=True)
    hist = hist.reshape((nbins,nbins,nbins))
    return hist


In [90]:
def CartesianProduct(*arrays):
    n = len(arrays)
    result = np.vstack(np.meshgrid(*arrays)).reshape(n,-1).T
    return result

In [91]:
hist_0 = LoadHist('H_0.txt')

hist_0=hist_0/hist_0.sum()
hist_1 = LoadHist('H_1.txt')
hist_1=hist_1/hist_1.sum()
it = 0
n = 1
sigma = 1
mu = np.zeros((n, 3))
mu[0][:] = 2

mu=np.reshape(mu,(3*n,))
print(mu)
hist = hist_0
r = np.arange(hist.shape[0])
c = CartesianProduct(r,r,r)

alfa = np.ones((n))
alfam = alfa*2
mum = np.reshape(mu, (3*n,))*2

[2. 2. 2.]


In [92]:
def f(hist,c, n,alpha, mu, sigma):

    #print(alpha)
    #print(mu)
    h = np.sum(hist)
    proba = 0
    for i in range(n):
        density = -((c-mu[i])**2).sum(axis=1)
        density = density/(2*(sigma**2))
        density = alpha[i]*np.exp(density)
        proba += density
    ret = ((h-proba)**2).sum()
    return ret

In [93]:
def grad_alfa(hist,c, n,alpha, mu, sigma):
    ret = np.zeros(n)
    local = np.zeros(n)
    r = c.shape[0]
    for j in range(r):
        for i in range(n):
            density = -((c[j]-mu[3*i:3*(i+1)])**2).sum()
            density = density/(2*(sigma**2))
            density = np.exp(density)
            local[i] = -density
        sumexp = (-alpha*local).sum()
        ret += 2*(hist[tuple(c[j])]-sumexp)*local
    return ret



In [94]:
#alfa es el alpha global y alpha el local
#mu es el glibal y mus el local

def grad_miu(hist,c, n,alpha, mu, sigma):
    ret = np.zeros(3*n)
    local1 = np.zeros(n)
    local2 = np.zeros(3*n)
    r = c.shape[0]
    for j in range(r):
        for i in range(n):
            density = -((c[j]-mu[3*i:3*(i+1)])**2).sum()
            density = density/(2*(sigma**2))
            density = np.exp(density)
            local2[3*i:3*(i+1)] = density*((alpha[i])/sigma**2)*(mu[3*i:3*(i+1)]-c[j])
            local1[i] = density
        sumexp = (-alpha*local1).sum()
        ret += 2*(hist[tuple(c[j])]+sumexp)*local2
    return ret


In [95]:

def backtracking(f, g, xk, dk, fk, gk, alpha0, c1, c2, max_iter=1000):
   
    alpha = 0
    beta = float('inf')
    alphai = alpha0
    it = 0
    while 1 and it < max_iter:
        if f(xk + alphai*dk) > fk + c1*alphai*np.dot(np.transpose(gk), dk):
            beta = alphai
            alphai = (alpha + beta)/2
        elif np.dot(np.transpose(g(xk + alphai*dk)), dk) < c2*np.dot(np.transpose(gk), dk):
            alpha = alphai
            if beta == float('inf'):
                alphai = 2*alpha
            else:
                alphai = (alpha + beta)/2
        else:
            break
        it+=1
    return alphai


def BB_paso(xk, xkm, gk, gkm, flag=False):
    s = xk - xkm
    y = gk - gkm
    if flag:
        return np.dot(np.transpose(s), s)/np.dot(np.transpose(s), y)
    return np.dot(np.transpose(s), y)/np.dot(np.transpose(y), y)
    
def BarzilaiBorwein(f, g, x0, tolx=1e-9, tolf=1e-9, tolg=1e-4, max_iter=10000):
    xk = x0
    xkm = xk + np.ones_like(xk)
    fk = f(x0)
    fkm = 2*fk
    gk = g(x0)
    #print(gk)
    k = 0
    while k < max_iter:
        if np.linalg.norm(fk - fkm)/max(1, np.linalg.norm(fk)) < tolf:
            print(np.linalg.norm(fk - fkm)/max(1, np.linalg.norm(fk)))
            print("tol f")
            break
        if np.linalg.norm(gk) < tolg:
            print("tol g")
            break
        if k == 0:
            #print(x0)
            alphak = backtracking(f, g, x0, -gk, fk, gk, 1, 0.001, 0.9)
        else:
            alphak = BB_paso(xk, xkm, gk, gkm, False)
        xkm = xk
        xk = xk - alphak*gk
        fkm = fk
        fk = f(xk)
        gkm = gk
        gk = g(xk)
        k += 1
        #print(xk)
    return xk

In [96]:
while it < 100:
    f_alpha=partial(f,hist,c,n,mu=mu,sigma=sigma)
    grad_alpha=partial(grad_alfa,hist,c,n,mu=mu,sigma=sigma)
    alfa = BarzilaiBorwein(f_alpha, grad_alpha, alfa, max_iter=1000)
    #print(alfa)
    #print(grad_alpha(alfa))
    grad_mu=partial(grad_miu,hist,c,n,alfa,sigma=sigma)
    f_mu=partial(f,hist,c,n,alfa,sigma=sigma)
   # print(mu)
    mu = BarzilaiBorwein(f_mu, grad_mu, np.reshape(mu, (3*n,)), max_iter=1000)
    #print(grad_mu(mum))
    if np.isclose(mum, mu).all() and np.isclose(alfam, alfa).all():
        break
    alfam = alfa
    mum = mu
    it+= 1

0.0
tol f
3.721357307751312e-16
tol f
0.0
tol f
3.7213573077513108e-16
tol f


In [97]:
alfa_0=alfa
mu_0=mu
print(alfa_0,mu_0)

[1.] [2. 2. 2.]


In [98]:
it = 0
n = 1
sigma = 1
mu = np.ones((n, 3))

mu=np.reshape(mu,(3*n,))
print(mu)
hist = hist_1
r = np.arange(hist.shape[0])
c = CartesianProduct(r,r,r)

alfa = np.ones((n))
alfam = alfa*2
mum = np.reshape(mu, (3*n,))*2

[1. 1. 1.]


In [102]:
while it < 1000:
    f_alpha=partial(f,hist,c,n,mu=mu,sigma=sigma)
    grad_alpha=partial(grad_alfa,hist,c,n,mu=mu,sigma=sigma)
    alfa = BarzilaiBorwein(f_alpha, grad_alpha, alfa, max_iter=1000)
    print(alfa)
    #print(grad_alpha(alfa))
    grad_mu=partial(grad_miu,hist,c,n,alfa,sigma=sigma)
    f_mu=partial(f,hist,c,n,alfa,sigma=sigma)
    print(mu)
    mu = BarzilaiBorwein(f_mu, grad_mu, np.reshape(mu,(3*n,)), max_iter=1000)
    #print(grad_mu(mum))
    if np.isclose(mum, mu).all() and np.isclose(alfam, alfa).all():
        break
    alfam = alfa
    mum = mu
    it+= 1

0.0
tol f
[1.]
[1. 1. 1.]
0.0
tol f


In [103]:
alfa_1=alfa
mu_1=mu

print(alfa_1,mu_1)

[1.] [1. 1. 1.]


In [54]:
def val_pixel(pixel, alpha, mu):
    val=0.0
    for i in range(len(alpha)):
        val+=alpha[i]*np.exp(-la.norm(pixel-mu[i])**2)/(2*sigma**2)
    return val

In [55]:
def clasifica_colores(pixel,alpha_1,mu_1,alpha_2,mu_2,eps):
    F_0=(val_pixel(pixel,alpha_1,mu_1)+eps)/(val_pixel(pixel,alpha_1,mu_1)+val_pixel(pixel,alpha_2,mu_2)+2*eps)
    F_1=(val_pixel(pixel,alpha_2,mu_2)+eps)/(val_pixel(pixel,alpha_1,mu_1)+val_pixel(pixel,alpha_2,mu_2)+2*eps)
    if F_0>=F_1:
        return True
    else:
        return False

In [57]:
import cv2 
img=cv2.imread('grave.png')
save=np.zeros_like(img)
print(save.shape)
for i in range (img.shape[0]):
    for j in range (img.shape[1]):
        pixel=img[i,j]/255
        pixel=pixel*hist_0.shape[0]
        pixel.astype(int)
        if clasifica_colores(pixel,alfa_0,mu_0,alfa_1,mu_1,0.01):
            save.itemset((i,j,0),0)
            save.itemset((i,j,1),0)
            save.itemset((i,j,2),255)
        else:
            save.itemset((i,j,0),255)
            save.itemset((i,j,1),0)
            save.itemset((i,j,2),0)

(600, 450, 3)


In [58]:
cv2.imwrite("resultante_grave.png",save)

True

In [104]:
def histogram(pixel,hist_1,hist_0,eps):
    H_1=(hist_0[tuple(pixel)]+eps)/(hist_1[tuple(pixel)]+hist_0[tuple(pixel)]+2*eps)
    H_2=(hist_1[tuple(pixel)]+eps)/(hist_1[tuple(pixel)]+hist_0[tuple(pixel)]+2*eps)
    if H_1>=H_2:
        return True
    else:
        return False

In [105]:
img=cv2.imread('rose.png')
save=np.zeros_like(img)
print(save.shape)
posi=np.linspace(0,255,hist_0.shape[0]+1)
for i in range (img.shape[0]):
    for j in range (img.shape[1]):
        x=posi.searchsorted(img[i,j][0])-2
        y=posi.searchsorted(img[i,j][1])-2
        z=posi.searchsorted(img[i,j][2])-2
        if histogram((x,y,z),hist_1,hist_0,0.01):
            save.itemset((i,j,2),255)
        else:
            save.itemset((i,j,0),255)

(500, 500, 3)


In [106]:
cv2.imwrite("resultante_suave.png",save)

True